# Opciones

Una opción es un contrato derivado que otorga al comprador el **derecho, pero no la obligación**  de comprar o vender un activo subyacente en una fecha futura y a un precio pactado (strike).

- **Tipos principales**:
  - **Long Call**: derecho a **comprar** el subyacente al precio de ejercicio.
  - **Long Put**: derecho a **vender** el subyacente al precio de ejercicio.
  - **Short Call**: obligación de **vender** el subyacente al precio de ejercicio.
  - **Short Put**: obligación de **comprar** el subyacente al precio de ejercicio.

- **Estilo de ejercicio**:
  - **Europeas**: solo se pueden ejercer al vencimiento.
  - **Americanas**: se pueden ejercer en cualquier momento hasta el vencimiento.


## 2. Payoffs

- Long Call: $\max(S_T - K, 0) - C$
- Long Put: $\max(K - S_T, 0) - P$
- Short Call: $-\max(S_T - K, 0) + C$
- Short Put: $-\max(K - S_T, 0) + P$

donde $S_T$ es el precio del subyacente al vencimiento, $K$ el strike, $C$ es la prima pagada para entrar a una position long call y $P$ es la prima pagada para entrar a un long put

## 3. Métodos de valuación

1. **Black-Scholes-Merton (BSM)**: fórmula cerrada para opciones europeas.

Para una opción europea con subyacente con precio actual $S$, precio de ejercicio $K$, tasa libre de riesgo $r$, rendimiento por dividendos continuo $q$, volatilidad $\sigma$ y tiempo a vencimiento $T$ (en años), definimos:

\begin{align}
d_1 &= \frac{\ln(S/K) + (r - q + 0.5\sigma^2)T}{\sigma\sqrt{T}}, \\
d_2 &= d_1 - \sigma\sqrt{T}.
\end{align}

Precio **call**: $C = S e^{-qT} N(d_1) - K e^{-rT} N(d_2)$

Precio **put**: $P = K e^{-rT} N(-d_2) - S e^{-qT} N(-d_1)$

donde $N(\cdot)$ es la CDF de una Normal estándar.


2. **Árbol binomial (CRR)**: aproximación discreta válida para europeas y americanas.
3. **Paridad put–call**: relación entre calls y puts europeos:  
   $$ C - P = S_0 - strike e^{-rT} $$

In [65]:
import numpy as np
import pandas as pd
import scipy.stats as si

In [ ]:
class Option:
    def __init__(self, position:str, option_type:str, expire_date:pd.Timestamp, emition_date:pd.Timestamp, calendar_convention:str = 'Actual/360'):
        
        # Convertimos los parametros a tipo correspondiente
        self.position = position.strip().lower()
        assert self.position in ('long','short'), 'Position must be long or short'

        self.option_type = option_type.strip().lower()
        assert self.option_type in ('call','put'), 'Option_type must be call or put'

        # Convertimos las fechas a tipo datetime
        self.expire_date = pd.to_datetime(expire_date)
        self.emition_date = pd.to_datetime(emition_date)

        # Convencion de calendario
        self.calendar_convention = calendar_convention.strip().lower()


    def day_count_fraction(self,start_date: pd.Timestamp, end_date: pd.Timestamp):
        """
        Calcula el número de días y la fracción de año entre dos fechas
        según la convención de calendario especificada en el bono.

        Parámetros
        ----------
        start_date : pd.Timestamp
            Fecha inicial.
        end_date : pd.Timestamp
            Fecha final.

        Retorna
        -------
        days : int
            Número de días entre las fechas.
        fraction : float
            Fracción del año transcurrido según la convención.
        """
        
        # Validación
        if not isinstance(start_date, pd.Timestamp) or not isinstance(end_date, pd.Timestamp):
            raise TypeError("Las fechas deben ser de tipo pd.Timestamp")

        start_date = start_date.normalize()
        end_date = end_date.normalize()

        assert start_date < end_date, 'end date must be after star date'

        # Días reales entre fechas
        days = (end_date - start_date).days

        # Actual/Actual ISDA
        if self.calendar_convention == "actual/actual":

            # Calculamos la fraccion del año por segmento del año tomando en cuenta años biciestos
            fraction = 0.0
            current_date = start_date
            while current_date < end_date:
                year_end = pd.Timestamp(year=current_date.year, month=12, day=31)
                period_end = min(year_end, end_date)

                days_in_period = (period_end - current_date).days + 1
                days_in_year = 366 if current_date.is_leap_year else 365

                fraction += days_in_period / days_in_year

                current_date = period_end + pd.Timedelta(days=1)

            return days, fraction

        # --- Otras convenciones ---
        elif self.calendar_convention == "actual/360":
            fraction = days / 360

        elif self.calendar_convention == "actual/365":
            fraction = days / 365

        elif self.calendar_convention == "30/360":
            d1 = min(start_date.day, 30)
            d2 = min(end_date.day, 30) if start_date.day == 30 else end_date.day
            days = (end_date.year - start_date.year) * 360 + \
                    (end_date.month - start_date.month) * 30 + \
                    (d2 - d1)
            fraction = days / 360

        elif self.calendar_convention == "30/365":
            d1 = min(start_date.day, 30)
            d2 = min(end_date.day, 30) if start_date.day == 30 else end_date.day
            days = (end_date.year - start_date.year) * 365 + \
                    (end_date.month - start_date.month) * 30 + \
                    (d2 - d1)
            fraction = days / 365

        else:
            raise ValueError(f"Convención '{self.calendar_convention}' no reconocida")

        return days, fraction
    
    @staticmethod
    def _ensure_decimal_rate(rate):
        """
        Convierte tasas a decimales. Soporta float, list, numpy array o pandas Series.
        """
        if isinstance(rate, pd.Series):
            return rate.apply(lambda x: x/100 if x > 1 else x)
        elif isinstance(rate, (list, np.ndarray)):
            return pd.Series(rate).apply(lambda x: x/100 if x > 1 else x).values
        elif isinstance(rate, (int, float)):
            return rate/100 if rate > 1 else rate
        else:
            raise TypeError("Rate must be float, int, list, numpy array, or pandas Series")
        

    def _black_scholes_pricer(self, spot_price:float, strike:float, risk_free_rate:float, sigma:float, valuation_date:pd.Timestamp, yield_rate:float=0):
        
        # Verificamos que las tasas sean correctas
        risk_free_rate = self._ensure_decimal_rate(risk_free_rate)
        yield_rate = self._ensure_decimal_rate(yield_rate)

        # Obtenemos el plazo a expiracion de acuerdo a la convencion del calendario
        days_to_expire, days_to_expire_as_year_fraction = self.day_count_fraction(valuation_date, self.expire_date)
        
        # Obtenemos la tasa de descuento, en caso de que yield_rate sea distinto de cero se trata de subyacente con dividendos, con retorno por conveniencia o de divisa
        risk_free_rate = risk_free_rate[days_to_expire] if isinstance(risk_free_rate, pd.Series) else risk_free_rate
        yield_rate = yield_rate[days_to_expire] if isinstance(yield_rate, pd.Series) else yield_rate

        d1 = (np.log(spot_price / strike) + (risk_free_rate - yield_rate + 0.5 * sigma**2) * days_to_expire_as_year_fraction) / (sigma * np.sqrt(days_to_expire_as_year_fraction))
        d2 = d1 - sigma * np.sqrt(days_to_expire_as_year_fraction)

        if self.option_type == "call":
            price = spot_price * np.exp(-yield_rate * days_to_expire_as_year_fraction) * si.norm.cdf(d1, 0.0, 1.0) - strike * np.exp(-risk_free_rate * days_to_expire_as_year_fraction) * si.norm.cdf(d2)
        elif self.option_type == "put":
            price = strike * np.exp(-risk_free_rate * days_to_expire_as_year_fraction) * si.norm.cdf(-d2, 0.0, 1.0) - spot_price * np.exp(-yield_rate * days_to_expire_as_year_fraction) * si.norm.cdf(-d1)

        return price
    
    
    def _binomial_pricer(self):
        pass


    def valuate(self, spot_price:float, strike:float, risk_free_rate:float|pd.Series, yield_rate:float|pd.Series = 0, valuation_date:pd.Timestamp = pd.Timestamp.today(), method:str='black-scholes', sigma:float=None):
        
        self.spot_price = spot_price
        self.strike = strike
        self.risk_free_rate = self._ensure_decimal_rate(risk_free_rate)
        self.yield_rate = self._ensure_decimal_rate(yield_rate)
        self.sigma = sigma
        self.valuation_date = pd.to_datetime(valuation_date) 

        # Obtenemos el plazo a expiracion de acuerdo con la convencion del calendario
        self.days_to_expire, self.days_to_expire_as_year_fraction = self.day_count_fraction(valuation_date, self.expire_date)
        
        # Obtenemos la tasa de descuento, en caso de que yield_rate sea distinto de cero se trata de subyacente con dividendos, con retorno por conveniencia o de divisa
        risk_free_rate = self.risk_free_rate[self.days_to_expire] if isinstance(self.risk_free_rate, pd.Series) else self.risk_free_rate
        yield_rate = self.yield_rate[self.days_to_expire] if isinstance(self.yield_rate, pd.Series) else self.yield_rate

        if method == 'black-scholes':
            self.valuation = self._black_scholes_pricer(self.spot_price, self.strike, risk_free_rate, self.sigma, self.valuation_date, yield_rate)

        return self.valuation
    
    def compute_sensibilities(self, spot_price:float, strike:float, risk_free_rate:float, yield_rate:float, sigma:float, d1:float, d2:float, valuation_date:pd.Timestamp):
        
        # Verificamos que las tasas sean correctas
        risk_free_rate = self._ensure_decimal_rate(risk_free_rate)
        yield_rate = self._ensure_decimal_rate(yield_rate)

        # Obtenemos el plazo a expiracion de acuerdo a la convencion del calendario
        days_to_expire, days_to_expire_as_year_fraction = self.day_count_fraction(valuation_date, self.expire_date)

        # Delta
        if self.option_type == "call":
            delta = np.exp(-yield_rate * days_to_expire_as_year_fraction) * si.norm.cdf(d1)
        elif self.option_type == "put":
            delta = np.exp(-yield_rate * days_to_expire_as_year_fraction) * (si.norm.cdf(d1) - 1)

        # Gamma
        gamma = (np.exp(-yield_rate * days_to_expire_as_year_fraction) * si.norm.pdf(d1)) / (spot_price * sigma * np.sqrt(days_to_expire_as_year_fraction))

        # Vega
        vega = spot_price * np.exp(-yield_rate * days_to_expire_as_year_fraction) * si.norm.pdf(d1) * np.sqrt(days_to_expire_as_year_fraction)

        # Theta
        if self.option_type == "call":
            theta = -(spot_price * np.exp(-yield_rate*days_to_expire_as_year_fraction) * si.norm.pdf(d1) * sigma) / (2 * np.sqrt(days_to_expire_as_year_fraction)) \
                    - risk_free_rate * strike * np.exp(-risk_free_rate*days_to_expire_as_year_fraction) * si.norm.cdf(d2) \
                    + yield_rate * spot_price * np.exp(-yield_rate*days_to_expire_as_year_fraction) * si.norm.cdf(d1)
        else:
            theta = -(spot_price * np.exp(-yield_rate*days_to_expire_as_year_fraction) * si.norm.pdf(d1) * sigma) / (2 * np.sqrt(days_to_expire_as_year_fraction)) \
                    + risk_free_rate * strike * np.exp(-risk_free_rate*days_to_expire_as_year_fraction) * si.norm.cdf(-d2) \
                    - yield_rate * spot_price * np.exp(-yield_rate*days_to_expire_as_year_fraction) * si.norm.cdf(-d1)

        # Rho
        if self.option_type == "call":
            rho = strike * days_to_expire_as_year_fraction * np.exp(-risk_free_rate*days_to_expire_as_year_fraction) * si.norm.cdf(d2)
        else:
            rho = -strike * days_to_expire_as_year_fraction * np.exp(-risk_free_rate*days_to_expire_as_year_fraction) * si.norm.cdf(-d2)

        self.sensibilities = {
            "Delta": delta,
            "Gamma": gamma,
            "Vega": vega,
            "Theta": theta,
            "Rho": rho
        }

        return self.sensibilities
    

    def proffit_and_losses(self, ST: float | np.ndarray):
        """
        Calcula la ganancia/pérdida neta (profit & loss) de la opción 
        en el vencimiento para un precio final ST del subyacente.

        Parámetros
        ----------
        ST : float o np.ndarray
            Precio del subyacente al vencimiento.

        Retorna
        -------
        profit : float o np.ndarray
            Ganancia/pérdida neta según posición (long/short) y tipo (call/put).
        """
        # Payoff según tipo de opción
        if self.option_type == "call":
            payoff = np.maximum(ST - self.strike, 0)
        elif self.option_type == "put":
            payoff = np.maximum(self.strike - ST, 0)

        # Ajuste según posición
        if self.position == "long":
            profit = payoff - self.valuation  # paga la prima
        elif self.position == "short":
            profit = self.valuation - payoff  # recibe la prima

        return profit


In [92]:
# Ejemplo de uso:
valuation_date = pd.Timestamp.today()

emition_date = pd.Timestamp.today()
days_to_expire = int(0.75 * 365)  # 273 días
expire_date = valuation_date + pd.Timedelta(days=days_to_expire)

print(expire_date)

spot_price = 100
strike = 105
rate = 0.05
sigma = 0.2
yield_rate = 0
position = 'long'
calendar_convention='Actual/actual'


call_option = Option('long', 'call', expire_date, emition_date, calendar_convention)
put_option = Option('long', 'put', expire_date, emition_date, calendar_convention)


call_option_price = call_option.valuate(spot_price, strike, rate, yield_rate, valuation_date, sigma=sigma)
put_option_price = put_option.valuate(spot_price, strike, rate, yield_rate, valuation_date, sigma=sigma)

print(f'Call option price ={call_option_price:.6f}')
print(f'Put option price ={put_option_price:.6f}')

parity_check = call_option_price - put_option_price - (spot_price * np.exp(-yield_rate * call_option.days_to_expire_as_year_fraction) - strike * np.exp(-rate*call_option.days_to_expire_as_year_fraction))

assert parity_check <= 1e-8, "put-call parity is violated!"

2026-05-30 09:23:24.698643
Call option price =6.354658
Put option price =7.514319


In [93]:
call_option.days_to_expire_as_year_fraction

0.7452054794520548